In [1]:
# Paths
DATA_PATH = r"..\data\processed\processed_1.pkl"

In [2]:
TECH_COLS = ['LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 
             'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 
             'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith']

TECH_NEXT_COLS = ['LanguageWantToWorkWith', 'DatabaseWantToWorkWith', 
                  'PlatformWantToWorkWith', 'WebframeWantToWorkWith', 
                  'MiscTechWantToWorkWith', 'ToolsTechWantToWorkWith']

ROLE_COLS = ['DevType']

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder


In [4]:
processed_df = pd.read_pickle(DATA_PATH)

In [5]:
df = processed_df.copy()

In [6]:
# One hot encoding
encoded_dfs = {}
for col in TECH_COLS :
    mlb = MultiLabelBinarizer()
    one_hot_encoded = pd.DataFrame(mlb.fit_transform(df[col]), columns=mlb.classes_, index=df.index)
    encoded_dfs[col] = one_hot_encoded
df = pd.concat( encoded_dfs, axis=1)


In [7]:
role_col = processed_df['DevType']
one_hot_single_col = pd.get_dummies(role_col)


In [8]:
multi_level_columns = pd.MultiIndex.from_product([['DevType'], one_hot_single_col.columns])


In [9]:
one_hot_single_col.columns = multi_level_columns


In [10]:
df = pd.concat([df, one_hot_single_col], axis=1)


In [16]:
skill_freq = df.drop('DevType', axis=1).sum().reset_index()       
skill_freq.columns = ['Group', 'Skill', 'Freq']
skill_freq.head()

C:\Users\Mohamed Gad\AppData\Local\Temp\ipykernel_19444\1766006599.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  skill_freq = df.drop('DevType', axis=1).sum().reset_index()


,Group,Skill,Freq
0,LanguageHaveWorkedWith,,2044
1,LanguageHaveWorkedWith,APL,225
2,LanguageHaveWorkedWith,Ada,677
3,LanguageHaveWorkedWith,Apex,579
4,LanguageHaveWorkedWith,Assembly,4753


In [15]:
skills_df.sum().reset_index()

,level_0,level_1,0
0,LanguageHaveWorkedWith,,2044
1,LanguageHaveWorkedWith,APL,225
2,LanguageHaveWorkedWith,Ada,677
3,LanguageHaveWorkedWith,Apex,579
4,LanguageHaveWorkedWith,Assembly,4753
...,...,...,...
232,ToolsTechHaveWorkedWith,liblittletest,37
233,ToolsTechHaveWorkedWith,npm,39609
234,ToolsTechHaveWorkedWith,pnpm,5036
235,ToolsTechHaveWorkedWith,snitch,55
